In [ ]:
import numpy as np
import math

In [ ]:
# scara
d6 = 0.05 (# wrist is added)
def scara_invkin(x,y,z,d1,d2):
    # using formulae from the textbook
    r = abs((x**2+y**2-d1**2-d2**2)/(2*d1*d2))
    theta2 = np.arctan(np.sqrt(abs(1-r**2))/r)
    theta1 = np.arctan(y/x) - np.arctan((d2*np.sin(theta2))/(d1+d2*np.cos(theta2)))
    d3 = -z - d6
    # converting from radians to degrees
    theta1_deg = theta1*180/np.pi
    theta2_deg = theta2*180/np.pi
    print("\nInverse kinematics solution:")
    print(f"theta1 = {theta1_deg} deg")
    print(f"theta2 = {theta2_deg} deg")
    return theta1, theta2

# standford
def stanford_inverse_kinematics(endeffector_position,lengthsoflinks):
    theta1 = np.arctan(endeffector_position[1]/endeffector_position[0])
    r = np.sqrt(endeffector_position[0]**2 + endeffector_position[1]**2)
    s = endeffector_position[2] - lengthsoflinks[0]
    theta2 = np.arctan(s/r)
    d3 = np.sqrt(r**2 + s**2) - lengthsoflinks[1]
    print("First Solution: \n", "Theta1 = ", theta1, "\n Theta2 =", theta2,"\n Extension: ", d3, "\n")
    print("Second Solution: \n", "Theta1 = ", np.pi + theta1, "\n Theta2 =", np.pi - theta2,"\n Extension:", d3)
    return theta1, theta2,d3

# inverse kinematics for puma
def puma_inverse_kinematics(l1,l2,l3,xc,yc,zc):
    theta1 = math.atan2(yc,xc)
    D = (xc*xc+yc*yc+(zc-l1)*(zc-l1)-l2*l2-l3*l3)/(2*l2*l3)
    if D>=1 or D<=-1:
        print("singular configuration")
    if D>1 or D<-1:
        print("outside workspace")   
    theta3 = (math.atan2((-math.sqrt(1-D*D)),D))
    theta2 = math.atan2(l1-zc,(math.sqrt(xc*xc+yc*yc)))-math.atan2((l3*math.sin(theta3)),(l2+l3*math.cos(theta3)))
    print("Theta1 = ",np.rad2deg(theta1),"\nTheta2 = ", np.rad2deg(theta2), "\nTheta3 = ", np.rad2deg(theta3))

    return theta1,theta2,theta3

def atransformation(LinkParameters):
  a = LinkParameters[0]
  d = LinkParameters[2]
  alpha = LinkParameters[1]
  theta = LinkParameters[3]
  A = np.array([[[np.cos(theta), -np.sin(theta) * np.cos(alpha), np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
                  [np.sin(theta), np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
                  [0, np.sin(alpha), np.cos(alpha), d],
                  [0, 0, 0, 1]]])
  return (A)

# Homogenous Matrices Forward Kinematics from DH
def forward_kinematics_homogenous_matrix(DH):
    A = np.identity(4)
    n = np.shape(DH)[0]

    for i in range(n):
        Anext = atransformation(DH[i])
        A = np.matmul(A,Anext)

    return A

class Robot():
  def __init__(self,robot_type):
    self.robot_type = robot_type
  
  def calc_invk_and_fk(self):
    if self.robot_type == 'SCARA':
      print("Enter the length of links d1 and d2:")
      d1 = float(input("d1 = "))
      d2 = float(input("d2 = "))
      n = int(input("Enter the no. of end effector points:"))
      for i in range(n):
        print(f"Enter x, y, z cordinate of point {i+1}:")
        x = float(input("x = "))
        y = float(input("y = "))
        z = float(input("z = "))
        theta1, theta2 = scara_invkin(x,y,z,d1,d2)
       
    elif self.robot_type == "STANFORD":
      print("Enter the length of links d1 and d2:")
      d1 = float(input("d1 = "))
      d2 = float(input("d2 = "))
      n = int(input("Enter the no. of end effector points:"))
      for i in range(n):
        print(f"Enter x, y, z cordinate of point {i+1}:")
        x = float(input("x = "))
        y = float(input("y = "))
        z = float(input("z = "))
        print("\nInverse kinematics solution: ")
        theta1, theta2, d3 = stanford_inverse_kinematics((x,y,z),(d1,d2))
        
    elif self.robot_type == "PUMA":
      print("Enter the length of links l1, l2 and l3:")
      l1 = float(input("l1 = "))
      l2 = float(input("l2 = "))
      l3 = float(input("l3 = "))
      n = int(input("Enter the no. of end effector points:"))
      for i in range(n):
        print(f"Enter x, y, z cordinate of point {i+1}:")
        x = float(input("x = "))
        y = float(input("y = "))
        z = float(input("z = "))
        print("\nInverse kinematics solution: ")
        theta1, theta2, theta3 = puma_inverse_kinematics(l1, l2, l3, x, y, z)
        print("\nforward kinematics solution")
        #LinkParameters = [a, alpha, d, theta]
        LinkParameters = [[l1, 0, 0, theta1], [l2, -np.pi/2, 0, theta2], [0, 0, 0, theta3]]
        T = forward_kinematics_homogenous_matrix(LinkParameters)
        print("Final transformation matrix: ")
        print(T)

SyntaxError: ignored

In [ ]:
"""
Inputs: 
A = (0.40, 0.06, 0.1)
B = (0.40, 0.01, 0.1)
C = (0.35, 0.01, 0.1)
D = (0.35, 0.06, 0.1)

"""
# output for scara robot
scara = Robot("SCARA")

In [ ]:
scara.calc_invk_and_fk()

Enter the length of links d1 and d2:
d1 = 0.25
d2 = 0.25
Enter the no. of end effector points:4
Enter x, y, z cordinate of point 1:
x = 0.40
y = 0.06
z = 0.1

Inverse kinematics solution:
theta1 = -27.475770490137332 deg
theta2 = 72.01307220017092 deg

Forward kinematics solution:
Final transformation matrix: 
[[[ 2.73969219e-01  4.61373469e-01 -8.43845596e-01  2.90293810e-01]
  [-1.42472127e-01  8.87206020e-01  4.38824761e-01 -1.50961399e-01]
  [ 9.51126995e-01 -4.23237934e-17  3.08800000e-01  2.37781749e-01]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]]
Enter x, y, z cordinate of point 2:
x = 0.40
y = .01
z = 0.1

Inverse kinematics solution:
theta1 = -35.4139253154836 deg
theta2 = 73.6920429992965 deg

Forward kinematics solution:
Final transformation matrix: 
[[[ 2.28848344e-01  5.79479266e-01 -7.82197044e-01  2.60958832e-01]
  [-1.62717778e-01  8.14986982e-01  5.56164674e-01 -1.85549261e-01]
  [ 9.59766305e-01 -4.40382989e-17  2.80800000e-01  2.39941576e-01

In [ ]:
stanford = Robot("STANFORD")
stanford.calc_invk_and_fk()

Enter the length of links d1 and d2:
d1 = 0.25
d2 = 0.25
Enter the no. of end effector points:4
Enter x, y, z cordinate of point 1:
x = 0.40
y = 0.06
z = 0.01

Inverse kinematics solution: 
First Solution: 
 Theta1 =  0.14888994760949725 
 Theta2 = -0.5355241819768428 
 Extension:  0.22031904065219388 

Second Solution: 
 Theta1 =  3.2904826011992903 
 Theta2 = 3.677116835566636 
 Extension: 0.22031904065219388

Forward kinematics solution:
Final transformation matrix: 
[[[ 8.50486511e-01  1.48340453e-01  5.04646217e-01  0.00000000e+00]
  [ 1.27572977e-01 -9.88936353e-01  7.56969326e-02  0.00000000e+00]
  [ 5.10291907e-01 -1.13892230e-16 -8.60001262e-01  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]]
Enter x, y, z cordinate of point 2:
x = 0.40
y = 0.01
z = 0.1

Inverse kinematics solution: 
First Solution: 
 Theta1 =  0.024994793618920156 
 Theta2 = -0.3586679747243447 
 Extension:  0.17731721238443 

Second Solution: 
 Theta1 =  3.16658744720871

In [ ]:
puma = Robot("PUMA")
puma.calc_invk_and_fk()

Enter the length of links l1, l2 and l3:
l1 = 0.25
l2 = 0.25
l3 = 0.25
Enter the no. of end effector points:4
Enter x, y, z cordinate of point 1:
x = 0.40
y = 0.06
z = 0.1

Inverse kinematics solution: 
Theta1 =  8.530765609948132 
Theta2 =  50.71648614651107 
Theta3 =  -60.73826046718986

forward kinematics solution
Final transformation matrix: 
[[[ 2.49940210e-01  4.46085940e-01 -8.59381886e-01  3.75067665e-01]
  [ 4.20065866e-01  7.49721211e-01  5.11334308e-01  2.51930585e-01]
  [ 8.72395873e-01 -4.88800000e-01  6.12323400e-17  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]]
Enter x, y, z cordinate of point 2:
x = 0.40
y = 0.01
z = 0.1

Inverse kinematics solution: 
Theta1 =  1.4320961841646465 
Theta2 =  51.83079015007421 
Theta3 =  -62.561257903728745

forward kinematics solution
Final transformation matrix: 
[[[ 2.75624929e-01  5.30855530e-01 -8.01388361e-01  3.99458006e-01]
  [ 3.69279757e-01  7.11235380e-01  5.98144376e-01  2.06595138e-01]
